In [ ]:
output_string = """
1140
forward 0 4.5
rotate 0 3.14159
forward 2 5
forward 3 -5
sell 1
buy 1
OK
"""
#前进，机器人ID，速度米/秒|旋转弧度/秒|

In [79]:
from frame import Map, parse_input
input_string = """
1144 199346
9
1 43.75 49.25 0 12 1
2 45.75 49.25 0 0 1
3 47.75 49.25 0 0 1
4 43.75 47.25 -1 0 0
5 45.75 47.25 168 0 0
6 47.75 47.25 -1 0 0
7 44.75 45.25 -1 0 0
8 46.75 45.25 -1 0 0
9 46.25 42.25 -1 0 0
5 3 0.9657950401 1 0 0 0 -0.3755806088 47.5760498 47.40252686
-1 0 0 0 0 0 0 -0.006108176429 43.75140762 48.23157501
-1 0 0 0 0 0 0 0 3.25 2.25
-1 0 0 0 0 0 0 0 45.75 1.75
OK
"""
## 最后的 4 行机器人数据：|所在工作台ID|携带物品类型|时间价值系数|碰撞价值系数|角速度|线速度|朝向|坐标
map_obj = parse_input(input_string)


In [80]:
def print_map(map_obj):# 遍历并打印Map对象的所有属性
    for w in map_obj.workbenches:
        for attr in dir(w):
            if not callable(getattr(w, attr)) and not attr.startswith("__"):
                print(f"workbenches对象属性 {attr}: {getattr(w, attr)}")

    # 遍历并打印Agent对象的所有属性
    for r in map_obj.robots:
        for attr in dir(r):
            if not callable(getattr(r, attr)) and not attr.startswith("__"):
                print(f"Agent对象属性 {attr}: {getattr(r, attr)}")

In [81]:
print_map(map_obj)

workbenches对象属性 coord: [43.75 49.25]
workbenches对象属性 material_state: [2, 3]
workbenches对象属性 product_state: True
workbenches对象属性 remaining_time: 0
workbenches对象属性 type_id: 1
workbenches对象属性 coord: [45.75 49.25]
workbenches对象属性 material_state: []
workbenches对象属性 product_state: True
workbenches对象属性 remaining_time: 0
workbenches对象属性 type_id: 2
workbenches对象属性 coord: [47.75 49.25]
workbenches对象属性 material_state: []
workbenches对象属性 product_state: True
workbenches对象属性 remaining_time: 0
workbenches对象属性 type_id: 3
workbenches对象属性 coord: [43.75 47.25]
workbenches对象属性 material_state: []
workbenches对象属性 product_state: False
workbenches对象属性 remaining_time: -1
workbenches对象属性 type_id: 4
workbenches对象属性 coord: [45.75 47.25]
workbenches对象属性 material_state: []
workbenches对象属性 product_state: False
workbenches对象属性 remaining_time: 168
workbenches对象属性 type_id: 5
workbenches对象属性 coord: [47.75 47.25]
workbenches对象属性 material_state: []
workbenches对象属性 product_state: False
workbenches对象属性 remaining_time: -1
wo

In [84]:
from matplotlib import pyplot as plt
import numpy as np
from frame import Map, parse_input

map_txt_file_path = '/mnt/e/2023huawei/LinuxRelease/LinuxRelease/maps/1.txt'
def read_map():
    # 打开文件并读取每一行，并对每一行进行映射转换
    with open(map_txt_file_path, "r") as f:
        ori_matrix = np.array([list(line.strip()) for line in f])
    search_chars = ['A', '1', '2', '3', '4', '5', '6', '7','8']
    char_to_coord = {}
    # 查找符合条件的元素的下标，并将下标和对应的值储存在字典中
    for char in search_chars:
        char_to_coord[char] = {}
        indices = np.where(ori_matrix == char)
        for i in range(len(indices[0])):
            char_to_coord[char][i] = (indices[1][i]*0.5+0.25, 50-(indices[0][i]*0.5+0.25))
    return char_to_coord

def w_v_fun(delta_dir,distance):
    k_w = 1 # 可以调节的参数
    w = k_w * delta_dir
    if distance >= 6:
        v = 6
    else:
        v = distance
    k_v = 1
    v = k_v*(1-(np.absolute(w)/np.pi))*v
    return w, v

def kinema(current_direct,current_coord,target_coord):
    distance = np.linalg.norm(target_coord - current_coord)
    tar_cur_dir = [target_coord[0]-current_coord[0],target_coord[1]-current_coord[1]]
    tar_dir = np.arctan2(tar_cur_dir[1],tar_cur_dir[0]) # 每帧更新
    delta_dir = tar_dir - current_direct
    if np.absolute(delta_dir) > np.pi : delta_dir += 2*np.pi # w的范围是[-pi,pi]
    print('cur_dir=',current_direct,'tar_dir=',tar_dir,'delta_dir=',delta_dir,'distance=',distance)
    w, v = w_v_fun(delta_dir = delta_dir, distance = distance)
    return w,v
    
def get_robot_info(map_obj):
    # 遍历并打印Agent对象的所有属性
    for r in map_obj.robots:
        # for attr in dir(r):
            # if not callable(getattr(r, attr)) and not attr.startswith("__"):
        # print(f"Agent对象属性 {attr}: {getattr(r, attr)}")
        print(getattr(r, 'coord'))

    
if __name__ == '__main__':
    # 读取地图txt获得各物体的坐标
    all_coord = read_map()
    print(all_coord['A'])
    map_obj = parse_input(input_string)
    # 假设传入目前点和一个目标点
    cur_point = np.array([10,10])
    tar_point = np.array([5,10])
    cur_direct = np.pi
    robot_info = get_robot_info(map_obj)
    w,v = kinema(cur_direct, cur_point, tar_point)
    
    print('w=',w,'v=',v)
    # cur_dir = np.pi # 每帧更新
    # tar_cur_dir = [points[1][0]-points[0][0],points[1][1]-points[0][1]]
    # tar_dir = np.arctan2(tar_cur_dir[1],tar_cur_dir[0]) # 每帧更新
    # delta_dir = tar_dir - cur_dir
    # if np.absolute(delta_dir) > np.pi : delta_dir += 2*np.pi # w的范围是[-pi,pi]
    # print('cur_dir=',cur_dir,'tar_dir=',tar_dir,'delta_dir=',delta_dir)
    # w, v = w_v_fun(delta_dir = delta_dir)
    

{0: (24.75, 38.75), 1: (24.75, 36.75), 2: (24.75, 34.25), 3: (24.75, 29.25)}
[47.5760498  47.40252686]
[43.75140762 48.23157501]
[3.25 2.25]
[45.75  1.75]
cur_dir= 3.141592653589793 tar_dir= 3.141592653589793 delta_dir= 0.0 distance= 5.0
w= 0.0 v= 5.0
